In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data_train = pd.read_csv('../data/salary.train.processed.csv').set_index('id')
data_test = pd.read_csv('../data/salary.test.processed.csv').set_index('id')

In [ ]:
import pandas as pd

# Example: assuming data_train is your processed dataset
X = data_train.drop(columns=['label'])
y = data_train['label']

In [ ]:

corr_with_label = X.corrwith(y).abs().sort_values(ascending=False)

print("Top correlated features with label:")
print(corr_with_label.head(10))

In [ ]:
threshold = 0.1
selected_features = corr_with_label[corr_with_label >= threshold].index.tolist()

In [ ]:
X_selected = X[selected_features]
print(f"Selected {len(selected_features)} features based on correlation >= {threshold}")

In [ ]:
corr_matrix = X_selected.corr().abs()

# Create upper triangle mask
upper = corr_matrix.where(~np.tril(np.ones(corr_matrix.shape)).astype(bool))

# Drop features with high inter-correlation
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]

X_final = X_selected.drop(columns=to_drop)
print(f"Dropped {len(to_drop)} redundant features.")

In [ ]:
print(to_drop)

In [ ]:
import pandas
import seaborn as sns
import matplotlib.pyplot as plt

# คำนวณ correlation matrix (เฉพาะ numeric features)
corr_matrix = data_train.corr(numeric_only=True)

# สร้าง heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(
    corr_matrix,
    annot=False,  # ไม่แสดงตัวเลขบน heatmap
    cmap='coolwarm',
    center=0,
    linewidths=0.5,  # ความหนาของเส้นแบ่ง
    linecolor='white'  # สีของเส้นแบ่ง
)
plt.title("Correlation Heatmap of All Features and Target")
plt.show()

In [ ]:
print(selected_features)

In [ ]:
pip install optuna

In [ ]:
pip install lightgbm

In [ ]:
data_train = pd.read_csv('../data/salary.train.processed.csv').set_index('id')
data_test = pd.read_csv('../data/salary.test.processed.csv').set_index('id')
data_train = data_train[X_final.columns.tolist() + ['label']]
data_test = data_test[X_final.columns.tolist() + ['label']]

In [ ]:
import pandas
import lightgbm as lgbm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# --- Load Data ---
try:
    train_df = data_train
    test_df = data_test
except FileNotFoundError:
    print("Error: Could not find the processed CSV files.")
    raise

X_train = train_df.drop('label', axis=1)
y_train = train_df['label']
X_test = test_df.drop('label', axis=1)
y_test = test_df['label']
# --- End of Data Loading ---

# --- Train-Validation Split ---
X_train_final, X_val_final, y_train_final, y_val_final = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

# --- Define LightGBM Model with Default Parameters ---
lgbm_model = lgbm.LGBMClassifier(
    objective='binary',
    metric='binary_logloss',
    random_state=42,
    n_estimators=1000,
    learning_rate=0.1,  # ค่า default
    num_leaves=31,      # ค่า default
    max_depth=-1,       # ค่า default (ไม่จำกัดความลึก)
    min_child_samples=20,  # ค่า default
    subsample=1.0,      # ค่า default
    colsample_bytree=1.0,  # ค่า default
    verbose=-1
)

# --- Train the Model with Early Stopping ---
lgbm_model.fit(
    X_train_final,
    y_train_final,
    eval_set=[(X_val_final, y_val_final)],
    eval_metric='logloss',
    callbacks=[lgbm.early_stopping(50, verbose=False)]
)

# --- Evaluate on Test Set ---
y_pred_lgbm = lgbm_model.predict(X_test)

print("\nLightGBM Classification Report:")
print(classification_report(y_test, y_pred_lgbm, digits=6))

In [ ]:
data_train = pd.read_csv('../data/salary.train.processed.csv').set_index('id')
data_test = pd.read_csv('../data/salary.test.processed.csv').set_index('id')
data_train = data_train[X_final.columns.tolist() + ['label']]
data_test = data_test[X_final.columns.tolist() + ['label']]

In [ ]:
import pandas
import optuna
import numpy as np
import lightgbm as lgbm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
import warnings
import sklearn

# Suppress warnings
warnings.filterwarnings('ignore')

# --- Load Data ---
try:
    train_df = data_train
    test_df = data_test
except FileNotFoundError:
    print("Error: Could not find the processed CSV files.")
    raise

X_train = train_df.drop('label', axis=1)
y_train = train_df['label']
X_test = test_df.drop('label', axis=1)
y_test = test_df['label']
# --- End of Data Loading ---

# --- Step 1: Define the Objective Function ---
def objective_lgbm(trial):
    """
    Objective function with manual Cross-Validation and Early Stopping.
    """
    # 1. Define the search space for LightGBM
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'random_state': 42,
        'n_estimators': 1000,
        'verbose': -1,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 10, 50),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 30),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0)
    }
    
    # 2. Set up 3-Fold Cross-Validation
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = []

    # 3. Manually run the CV loop
    for train_idx, val_idx in skf.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        model = lgbm.LGBMClassifier(**param)
        
        # Train with early stopping
        model.fit(
            X_train_fold, 
            y_train_fold,
            eval_set=[(X_val_fold, y_val_fold)],
            eval_metric='logloss',
            callbacks=[lgbm.early_stopping(50, verbose=False)]
        )
        
        preds = model.predict(X_val_fold)
        f1 = sklearn.metrics.f1_score(y_val_fold, preds, average='binary')
        scores.append(f1)
    
    return np.mean(scores)

# --- Step 2: Create and Run the Study ---
study_name = "lgbm_salary_tuning"  # ตั้งชื่อโปรเจกต์ (แต่ไม่บันทึกลง storage)

print(f"Starting Optuna study: '{study_name}'")
print(f"Running 100 new trials...")

# สร้าง Study ใหม่โดยไม่ใช้ storage
study_lgbm = optuna.create_study(
    study_name=study_name,
    direction='maximize'
)

# รัน optimization
study_lgbm.optimize(objective_lgbm, n_trials=100)
print("Study complete!")
print(f"Total number of trials in study: {len(study_lgbm.trials)}")

# --- Step 3: Get Best Params ---
print("\nBest trial:")
print(f"  Value (Mean F1): {study_lgbm.best_value:.4f}")
print("  Best Params: ")
print(study_lgbm.best_params)

# เก็บพารามิเตอร์ที่ดีที่สุด
best_params = study_lgbm.best_params
print(f"\nSuccessfully stored parameters in 'best_params' variable.")

# --- Step 4: Train the FINAL Model ---
X_train_final, X_val_final, y_train_final, y_val_final = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

final_lgbm = lgbm.LGBMClassifier(
    random_state=42,
    n_estimators=1000,
    **best_params
)

final_lgbm.fit(
    X_train_final, 
    y_train_final,
    eval_set=[(X_val_final, y_val_final)],
    eval_metric='logloss',
    callbacks=[lgbm.early_stopping(50, verbose=False)]
)

# --- Step 5: Evaluate the Optuna-Tuned Model on the TEST set ---
y_pred_optuna_lgbm = final_lgbm.predict(X_test)

print("\nOptuna-Tuned LightGBM Classification Report:")
print(classification_report(y_test, y_pred_optuna_lgbm, digits=6))

In [ ]:
print(best_params)

n_estimators= 900, learning_rate= 0.05487718973019812, num_leaves:=63, min_child_samples 42, subsample=0.8406013365172204, colsample_bytree= 0.1319962821123755, reg_alpha= 0.995264535862729, reg_lambda= 0.01927049495328959,

In [ ]:
data_train = pd.read_csv('../data/salary.train.processed.csv').set_index('id')
data_test = pd.read_csv('../data/salary.test.processed.csv').set_index('id')
data_train = data_train[X_final.columns.tolist() + ['label']]
data_test = data_test[X_final.columns.tolist() + ['label']]

In [ ]:
import pandas
import lightgbm as lgb
import sklearn.metrics
from sklearn.metrics import classification_report

# --- 1. Load Data (to calculate the weight) ---
try:
    data_train_full = data_train
except FileNotFoundError:
    print("Error: ไม่พบไฟล์ salary.train.processed.csv กรุณาตรวจสอบ path")
    # exit() 

X_full = data_train_full.drop(['label'], axis='columns')
y_full = data_train_full['label']

# --- 2. Calculate scale_pos_weight ---
# This is the same logic as your XGBoost code
scale_pos_weight = len(y_full[y_full == 0]) / len(y_full[y_full == 1])
print(f"Calculated scale_pos_weight: {scale_pos_weight:.4f}")

# --- 3. Define Your Best Parameters ---
# These are the params you provided in your prompt
best_lgbm_params = best_params

# --- 4. Create and Train the Model ---
print("\nTraining LightGBM model with class weighting...")

lgbm_model_final = lgb.LGBMClassifier(
    **best_lgbm_params,       # Apply all your tuned parameters
    scale_pos_weight=scale_pos_weight, # 👈 Here is the class weight
    random_state=42,
    n_jobs=-1
)

# Train the model on the full training dataset
lgbm_model_final.fit(X_full, y_full)

print("Model training complete.")

# --- 5. Evaluate on Test Data (Recommended) ---
print("\nEvaluating model on test data...")

data_test_lgbm = data_test
data_test_lgbm['prediction'] = lgbm_model_final.predict(data_test_lgbm.drop(['label'], axis='columns'))

# Print the report
report_scores_lgbm = sklearn.metrics.classification_report(
    y_true=data_test_lgbm['label'],
    y_pred=data_test_lgbm['prediction'],
    digits=6,
    output_dict=True
)
df_score_lgbm = pandas.DataFrame(report_scores_lgbm).transpose()

print("\nLightGBM (Tuned + Weighted) Report:")
print(df_score_lgbm)

In [ ]:
data_train = pd.read_csv('../data/salary.train.processed.csv').set_index('id')
data_test = pd.read_csv('../data/salary.test.processed.csv').set_index('id')
data_train = data_train[X_final.columns.tolist() + ['label']]
data_test = data_test[X_final.columns.tolist() + ['label']]

In [ ]:
import json
import joblib
import pandas
import lightgbm as lgb
import sklearn.metrics
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE  # 👈 1. Import SMOTE

# --- 1. Load Data (Needed for training) ---
try:
    data_train_full = data_train
except FileNotFoundError:
    print("Error: ไม่พบไฟล์ salary.train.processed.csv กรุณาตรวจสอบ path")
    # exit() 

X_full = data_train_full.drop(['label'], axis='columns')
y_full = data_train_full['label']

print(f"Original training data shape: {X_full.shape}")
print(f"Original label distribution:\n{y_full.value_counts()}")

# --- 2. Define Your Best Parameters ---
# These are the params you provided
best_lgbm_params = best_params

# --- 3. Apply SMOTE to the Training Data ---
print("\nApplying SMOTE to the training data...")
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_full, y_full)

print(f"New resampled training data shape: {X_resampled.shape}")
print(f"New resampled label distribution:\n{y_resampled.value_counts()}")

# --- 4. Create and Train the Final Model (with NO class_weight) ---
print("\nTraining final LGBM model on SMOTEd data...")

lgbm_model_final = lgb.LGBMClassifier(
    **best_lgbm_params,
    # ⚠️ NO 'scale_pos_weight' or 'class_weight' here
    random_state=42,
    n_jobs=-1
)

# 5. Train the model on the NEW resampled data
lgbm_model_final.fit(X_resampled, y_resampled)
print("Model training complete.")

# --- 6. Evaluate on ORIGINAL Test Data ---
print("\nEvaluating model on *original* test data...")

data_test_lgbm = data_test

# IMPORTANT: Do NOT apply SMOTE to the test data.
X_test = data_test_lgbm.drop(['label'], axis='columns')
y_test = data_test_lgbm['label']

data_test_lgbm['prediction'] = lgbm_model_final.predict(X_test)

# Print the report
report_scores_lgbm = sklearn.metrics.classification_report(
    y_true=y_test,
    y_pred=data_test_lgbm['prediction'],
    digits=6,
    output_dict=True
)
df_score_lgbm = pandas.DataFrame(report_scores_lgbm).transpose()

print("\nLightGBM (Tuned + SMOTE) Report:")
print(df_score_lgbm)

In [ ]:
data_train = pd.read_csv('../data/salary.train.processed.csv').set_index('id')
data_test = pd.read_csv('../data/salary.test.processed.csv').set_index('id')
data_train = data_train[X_final.columns.tolist() + ['label']]
data_test = data_test[X_final.columns.tolist() + ['label']]

In [ ]:
import pandas
import lightgbm as lgb
import sklearn.metrics
from sklearn.metrics import classification_report
from imblearn.combine import SMOTETomek

print("\n--- 3. Testing LightGBM with SMOTETomek ---")

# --- Load Data ---
try:
    data_train_full = data_train
    data_test_lgbm = data_test
except FileNotFoundError:
    print("Error: ไม่พบไฟล์ salary.train.processed.csv กรุณาตรวจสอบ path")
    # exit() 

X_full = data_train_full.drop(['label'], axis='columns')
y_full = data_train_full['label']
X_test = data_test_lgbm.drop(['label'], axis='columns')
y_test = data_test_lgbm['label']

# --- Apply SMOTETomek ---
print("Applying SMOTETomek...")
smt = SMOTETomek(random_state=42, n_jobs=-1)
X_resampled, y_resampled = smt.fit_resample(X_full, y_full)
print(f"New resampled label distribution:\n{y_resampled.value_counts()}")

# --- Define Parameters ---
best_lgbm_params = best_params

# --- Create and Train Model ---
lgbm_model = lgb.LGBMClassifier(
    **best_lgbm_params,
    # ⚠️ NO 'scale_pos_weight' or 'class_weight'
    random_state=42,
    n_jobs=-1
)

lgbm_model.fit(X_resampled, y_resampled) # Train on SMOTETomek data
print("Model training complete.")

# --- Evaluate ---
y_pred = lgbm_model.predict(X_test)
report = classification_report(y_test, y_pred, digits=6, output_dict=True)
df_report = pandas.DataFrame(report).transpose()

print("\nLightGBM (Tuned + SMOTETomek) Report:")
print(df_report)

In [ ]:
data_train = pd.read_csv('../data/salary.train.processed.csv').set_index('id')
data_test = pd.read_csv('../data/salary.test.processed.csv').set_index('id')
data_train = data_train[X_final.columns.tolist() + ['label']]
data_test = data_test[X_final.columns.tolist() + ['label']]

In [ ]:
import pandas
import lightgbm as lgb
import sklearn.metrics
from sklearn.metrics import classification_report
from imblearn.over_sampling import ADASYN

print("\n--- 4. Testing LightGBM with ADASYN ---")

# --- Load Data ---
try:
    data_train_full = data_train
    data_test_lgbm = data_test
except FileNotFoundError:
    print("Error: ไม่พบไฟล์ salary.train.processed.csv กรุณาตรวจสอบ path")
    # exit() 

X_full = data_train_full.drop(['label'], axis='columns')
y_full = data_train_full['label']
X_test = data_test_lgbm.drop(['label'], axis='columns')
y_test = data_test_lgbm['label']

# --- Apply ADASYN ---
print("Applying ADASYN...")
ada = ADASYN(random_state=42)
X_resampled, y_resampled = ada.fit_resample(X_full, y_full)
print(f"New resampled label distribution:\n{y_resampled.value_counts()}")

# --- Define Parameters ---
best_lgbm_params = best_params

# --- Create and Train Model ---
lgbm_model = lgb.LGBMClassifier(
    **best_lgbm_params,
    # ⚠️ NO 'scale_pos_weight' or 'class_weight'
    random_state=42,
    n_jobs=-1
)

lgbm_model.fit(X_resampled, y_resampled) # Train on ADASYN data
print("Model training complete.")

# --- Evaluate ---
y_pred = lgbm_model.predict(X_test)
report = classification_report(y_test, y_pred, digits=6, output_dict=True)
df_report = pandas.DataFrame(report).transpose()

print("\nLightGBM (Tuned + ADASYN) Report:")
print(df_report)

In [ ]:
data_train = pd.read_csv('../data/salary.train.processed.csv').set_index('id')
data_test = pd.read_csv('../data/salary.test.processed.csv').set_index('id')
data_train = data_train[X_final.columns.tolist() + ['label']]
data_test = data_test[X_final.columns.tolist() + ['label']]

In [ ]:
import pandas
import lightgbm as lgb
import sklearn.metrics
from sklearn.metrics import classification_report
from imblearn.combine import SMOTETomek

print("\n--- 3. Testing LightGBM with SMOTETomek ---")

# --- Load Data ---
try:
    data_train_full = data_train
    data_test_lgbm = data_test
except FileNotFoundError:
    print("Error: ไม่พบไฟล์ salary.train.processed.csv กรุณาตรวจสอบ path")
    # exit() 

X_full = data_train_full.drop(['label'], axis='columns')
y_full = data_train_full['label']
X_test = data_test_lgbm.drop(['label'], axis='columns')
y_test = data_test_lgbm['label']

# --- Apply SMOTETomek ---
print("Applying SMOTETomek...")
smt = SMOTETomek(random_state=42, n_jobs=-1)
X_resampled, y_resampled = smt.fit_resample(X_full, y_full)
print(f"New resampled label distribution:\n{y_resampled.value_counts()}")

# --- Define Parameters ---
# final_best_lgbm_params = best_params

# --- Create and Train Model ---
best_lgbm_model = lgb.LGBMClassifier(
    n_estimators= 900, learning_rate= 0.05487718973019812, num_leaves=63, min_child_samples=42, subsample=0.8406013365172204, colsample_bytree=0.1319962821123755, reg_alpha=0.995264535862729, reg_lambda=0.01927049495328959,
    # ⚠️ NO 'scale_pos_weight' or 'class_weight'
    random_state=42,
    n_jobs=-1
)

best_lgbm_model.fit(X_resampled, y_resampled) # Train on SMOTETomek data
print("Model training complete.")

# --- Evaluate ---
y_pred = best_lgbm_model.predict(X_test)
report = classification_report(y_test, y_pred, digits=6, output_dict=True)
df_report = pandas.DataFrame(report).transpose()

print("\nLightGBM (Tuned + SMOTETomek) Report:")
print(df_report)

In [ ]:
print(best_lgbm_model.get_params())

In [ ]:
joblib.dump(best_lgbm_model, '../model/lgbm/lgbm_model_final_smote.joblib')
with open('../model/lgbm/lgbm_config.json','w')as f:
    json.dump(
        obj=best_lgbm_model.get_params(),
        fp=f,
        indent = 4
    )

{
    'reg_lambda': 0, 
    'reg_alpha': 0.1, 
    'num_leaves': 31, 
    'n_estimators': 200, 
    'max_depth': 30, 
    'learning_rate': 0.05,
    'random_state': 42, 
    'n_jobs': -1
}

In [ ]:
data_train = pd.read_csv('../data/salary.train.processed.csv').set_index('id')
data_test = pd.read_csv('../data/salary.test.processed.csv').set_index('id')
data_train = data_train[X_final.columns.tolist() + ['label']]
data_test = data_test[X_final.columns.tolist() + ['label']]

In [ ]:
{'n_estimators': 102, 'learning_rate': 0.26992371517621444, 'num_leaves': 22, 'max_depth': 5, 'reg_alpha': 8.715022016449311e-06, 'reg_lambda': 3.587506772725946, 'subsample': 0.9863285464013338, 'colsample_bytree': 0.7550416705324272}

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import sklearn.metrics
from sklearn.metrics import f1_score, precision_recall_curve, classification_report
import warnings

warnings.filterwarnings('ignore')

print("--- Training Final LightGBM Model with Specific Parameters ---")

# --- 1. โหลดข้อมูล ---
try:
    train_df = pd.read_csv("../data/salary.train.processed.csv")
    test_df = pd.read_csv("../data/salary.test.processed.csv")
    print("Files loaded successfully.")
except FileNotFoundError:
    print("Error: Files not found. Make sure 'salary.train.processed.csv' and 'salary.test.processed.csv' are in the same directory as the script.")
    exit()

# --- 2. คำนวณ Weight (We don't use it, but it's good to check) ---
label_counts = train_df['label'].value_counts()
scale_pos_weight = label_counts[0] / label_counts[1]
print(f"Data Check: scale_pos_weight = {scale_pos_weight:.4f}")

# --- 3. สร้างชุดข้อมูล Train/Test ---
columns_to_drop = ['id', 'social-security-number', 'house-number', 'fnlwgt']
target_column = 'label'

train_cols_to_drop_actual = [col for col in columns_to_drop if col in train_df.columns]
y_train = train_df[target_column]
X_train = train_df.drop(columns=train_cols_to_drop_actual + [target_column])

test_cols_to_drop_actual = [col for col in columns_to_drop if col in test_df.columns]
y_test = test_df[target_column]
X_test = test_df.drop(columns=test_cols_to_drop_actual + [target_column])

print("Data splits created.")

# --- 4. กำหนดโมเดลและ Parameters ที่คุณเลือก ---
final_params = {
    'n_estimators': 900,
    'learning_rate': 0.05487718973019812,
    'num_leaves': 63,
    'min_child_samples': 42,
    'subsample': 0.8406013365172204,
    'colsample_bytree': 0.1319962821123755,
    'reg_alpha': 0.995264535862729,
    'reg_lambda': 0.01927049495328959,
    'random_state': 42,
    'n_jobs': -1
    # 'scale_pos_weight' is intentionally removed
}

finalbest_lgbm_model = lgb.LGBMClassifier(**final_params)

print("\nStarting model training with specified parameters...")
# --- 5. เริ่มการ Train โมเดล ---
finalbest_lgbm_model.fit(X_train, y_train)

print("\nTraining complete!")
print("="*50)

# --- 6. ประเมินผลโมเดล (ที่ Default Threshold 0.5) ---
print("--- Results on Test Set (Default 0.5 Threshold) ---")
y_pred_default = finalbest_lgbm_model.predict(X_test)
print(classification_report(y_test, y_pred_default, digits=4))
print("="*50)

# --- 7. ค้นหา F1 ที่ดีที่สุด และแสดง Full Report ---
print("\n--- Finding Optimal Threshold for Best F1-Score ---")
y_pred_probs = finalbest_lgbm_model.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, y_pred_probs)
f1_scores = (2 * precision * recall) / (precision + recall)
f1_scores = np.nan_to_num(f1_scores)

best_f1_index = np.argmax(f1_scores[:-1])
best_f1 = f1_scores[best_f1_index]
best_threshold = thresholds[best_f1_index]

print(f"   Best Threshold Found: {best_threshold:.4f}")
print(f"   Best F1-Score (for class 1.0): {best_f1:.4f}")

# --- นี่คือส่วนที่เพิ่มเข้ามา ---
# 7b. สร้าง predictions โดยใช้ threshold ที่ดีที่สุด
y_pred_optimal = (y_pred_probs >= best_threshold).astype(int)

print("\n--- Full Report at Optimal Threshold ---")
# 7c. พิมพ์ classification report ฉบับเต็ม
print(classification_report(y_test, y_pred_optimal, digits=4))
# ---------------------------------

print("\n--- Script Finished ---")




In [ ]:
# 7c. พิมพ์ classification report ฉบับเต็ม
print(classification_report(y_test, y_pred_optimal, digits=4))

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
import warnings

warnings.filterwarnings('ignore')

print("--- Running the NEW Champion Model (F1 0.8105) ---")

# --- 1. โหลดข้อมูล ---
try:
    train_df = pd.read_csv("../data/salary.train.processed.csv")
    test_df = pd.read_csv("../data/salary.test.processed.csv")
    print("Files loaded successfully.")
except FileNotFoundError:
    print("Error: Files not found.")
    exit()

# --- 2. สร้างชุดข้อมูล Train/Test ---
columns_to_drop = ['id', 'social-security-number', 'house-number', 'fnlwgt']
target_column = 'label'

train_cols_to_drop = [col for col in columns_to_drop if col in train_df.columns]
y_train = train_df[target_column]
X_train = train_df.drop(columns=train_cols_to_drop + [target_column])

test_cols_to_drop = [col for col in columns_to_drop if col in test_df.columns]
y_test = test_df[target_column]
X_test = test_df.drop(columns=test_cols_to_drop + [target_column])

print("Data splits created.")

# --- 3. 🔑 "สูตร" ที่ดีที่สุด (The Full Recipe F1 0.8105) ---

# 3a. ค่าพารามิเตอร์โมเดล (จาก Optuna รอบล่าสุด)
best_params = {
    'n_estimators': 102,
    'learning_rate': 0.26992371517621444,
    'num_leaves': 22,
    'max_depth': 5,
    'reg_alpha': 8.715022016449311e-06,
    'reg_lambda': 3.587506772725946,
    'subsample': 0.9863285464013338,
    'colsample_bytree': 0.7550416705324272,
    
    # พารามิเตอร์คงที่ (Static) ที่ต้องใส่
    'random_state': 42,
    'n_jobs': -1
}

# 3b. ค่าถ่วงน้ำหนัก (แก้ข้อมูลไม่สมดุล)
# (ค่านี้มาจากข้อมูล ไม่ได้มาจาก Optuna แต่ถูกใช้ตอนเทรน)
# We calculate it just in case, to be precise
label_counts = y_train.value_counts()
scale_pos_weight = label_counts[0] / label_counts[1]
print(f"Calculated scale_pos_weight: {scale_pos_weight:.4f}")


# 3c. เกณฑ์การตัดสินใจ (Threshold) ที่ดีที่สุด
best_threshold = 0.4675

print(f"Using Best Threshold: {best_threshold}")

# --- 4. สร้างและเทรนโมเดล ---
print("Training final champion model...")
final_model_lgbm = lgb.LGBMClassifier(
    scale_pos_weight=scale_pos_weight, 
    **best_params
)

final_model_lgbm.fit(X_train, y_train)

# --- 5. ประเมินผลบน Test Set (โดยใช้ Threshold ที่ดีที่สุด) ---
print("Evaluating model...")
# 5a. ดึงค่าความน่าจะเป็น (Probability)
y_pred_probs = final_model_lgbm.predict_proba(X_test)[:, 1]

# 5b. ใช้ Threshold ที่ดีที่สุดในการตัดสินใจ
y_final_pred = (y_pred_probs >= best_threshold).astype(int)

# 5c. คำนวณผลลัพธ์
final_f1 = f1_score(y_test, y_final_pred)
final_recall = recall_score(y_test, y_final_pred)
final_precision = precision_score(y_test, y_final_pred)

print("\n" + "="*50)
print("--- Final NEW Champion Model Results ---")
print(f"   Best F1 Score:  {final_f1:.4f}")
print(f"   Recall:         {final_recall:.4f}")
print(f"   Precision:      {final_precision:.4f}")
print("="*50)

# --- 6. พิมพ์ classification report ฉบับเต็ม ---
print("\n--- Full Classification Report (at Optimal Threshold) ---")
print(classification_report(y_test, y_final_pred, digits=4))
# -------------------------------------------------

print("\n--- Script Finished ---")


--- Running the NEW Champion Model (F1 0.8105) ---
Files loaded successfully.
Data splits created.
Calculated scale_pos_weight: 1.3882
Using Best Threshold: 0.4675
Training final champion model...
[LightGBM] [Info] Number of positive: 7001, number of negative: 9719
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000722 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 125
[LightGBM] [Info] Number of data points in the train set: 16720, number of used features: 49
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.418720 -> initscore=-0.328030
[LightGBM] [Info] Start training from score -0.328030
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No

In [ ]:
print("\n--- Full Classification Report (at Optimal Threshold) ---")
print(classification_report(y_test, y_final_pred, digits=4))

In [ ]:
import joblib
import json
joblib.dump(final_model_lgbm, '../model/lgbm/lgbm_model_final_smote.joblib')
with open('../model/lgbm/lgbm_config.json','w')as f:
    json.dump(
        obj=final_model_lgbm.get_params(),
        fp=f,
        indent = 4
    )

In [ ]:
pip install category_encoders

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
from imblearn.over_sampling import SMOTE
import warnings

warnings.filterwarnings('ignore')

print("--- Running Optuna Search with CV (v5 - Improved) ---")

# --- 1. Load Data ---
try:
    train_df = pd.read_csv("../data/salary.train.processed.csv")
    test_df = pd.read_csv("../data/salary.test.processed.csv")
    print("Files loaded successfully.")
except FileNotFoundError:
    print("Error: Files not found. Make sure paths are correct.")
    exit()

# --- 2. Preprocess Data ---
columns_to_drop = ['id', 'social-security-number', 'house-number', 'fnlwgt']
target_column = 'label'

# 2a. Apply Target Encoding for Categorical Features
categorical_columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
if any(col in train_df.columns for col in categorical_columns):
    encoder = TargetEncoder(cols=[col for col in categorical_columns if col in train_df.columns])
    train_df = encoder.fit_transform(train_df, train_df[target_column])
    test_df = encoder.transform(test_df)

# 2b. Scale Numerical Features
numerical_columns = ['age', 'hours-per-week', 'capital-gain', 'capital-loss']
scaler = StandardScaler()
if any(col in train_df.columns for col in numerical_columns):
    train_df[numerical_columns] = scaler.fit_transform(train_df[numerical_columns])
    test_df[numerical_columns] = scaler.transform(test_df[numerical_columns])

# 2c. Prepare Data Splits
train_cols_to_drop = [col for col in columns_to_drop if col in train_df.columns]
y_train_full = train_df[target_column]  # with 'label'
X_train_full = train_df.drop(columns=train_cols_to_drop + [target_column]) # without label

test_cols_to_drop = [col for col in columns_to_drop if col in test_df.columns]
y_test = test_df[target_column]  # with 'label'
X_test = test_df.drop(columns=test_cols_to_drop + [target_column]) # without label

# 2d. Apply SMOTE
smote = SMOTE(random_state=42)
X_train_full, y_train_full = smote.fit_resample(X_train_full, y_train_full)


# 2e. Split for CV and Thresholding
X_train_cv, X_val_thresh, y_train_cv, y_val_thresh = train_test_split(
    X_train_full, y_train_full, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_train_full
)

print(f"Optuna CV set: {X_train_cv.shape[0]} samples")
print(f"Threshold-finding set: {X_val_thresh.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

# 2f. Calculate scale_pos_weight
label_counts = y_train_full.value_counts()
scale_pos_weight = label_counts[0] / label_counts[1]
print(f"Calculated scale_pos_weight: {scale_pos_weight:.4f}")

# --- 3. Define Optuna Objective Function ---
def objective(trial):
    params = {
        'objective': 'binary',
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'max_depth': trial.suggest_int('max_depth', 5, 30),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 3.0),
        'random_state': 42,
        'n_jobs': -1
    }
    
    max_n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    cv_folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    f1_scores = []
    
    for train_idx, val_idx in cv_folds.split(X_train_cv, y_train_cv):
        X_train_fold, X_val_fold = X_train_cv.iloc[train_idx], X_train_cv.iloc[val_idx]
        y_train_fold, y_val_fold = y_train_cv.iloc[train_idx], y_train_cv.iloc[val_idx]
        
        model = lgb.LGBMClassifier(**params, n_estimators=max_n_estimators)
        model.fit(
            X_train_fold, y_train_fold,
            eval_set=[(X_val_fold, y_val_fold)],
            eval_metric='auc',
            callbacks=[lgb.early_stopping(20, verbose=False)]
        )
        
        y_val_probs = model.predict_proba(X_val_fold)[:, 1]
        thresholds = np.arange(0.35, 0.50, 0.001)
        best_f1 = 0
        for thresh in thresholds:
            preds = (y_val_probs >= thresh).astype(int)
            f1 = f1_score(y_val_fold, preds)
            if f1 > best_f1:
                best_f1 = f1
        
        f1_scores.append(best_f1)
    
    mean_f1 = np.mean(f1_scores)
    trial.set_user_attr('best_n_estimators', model.best_iteration_)
    return mean_f1

# --- 4. Run Optuna Study ---
print("\n--- Starting Optuna optimization (this may take time)... ---")
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print("Optimization finished.")
print(f"Best trial (Mean CV F1): {study.best_value:.4f}")

# --- 5. Get Best Parameters ---
best_hyperparams = study.best_params
best_n_estimators = study.best_trial.user_attrs['best_n_estimators']

print(f"Found Optimal n_estimators: {best_n_estimators}")
print("Found Best Hyperparameters:")
for key, value in best_hyperparams.items():
    if key != 'n_estimators':
        print(f"  {key}: {value}")

# --- 6. Train Final Model ---
print("\n--- Training final model on ALL training data... ---")
final_params_for_model = {
    **best_hyperparams,
    'n_estimators': best_n_estimators,
    'objective': 'binary',
    'random_state': 42,
    'n_jobs': -1
}

final_model = lgb.LGBMClassifier(**final_params_for_model)
final_model.fit(X_train_full, y_train_full)

# --- 7. Find Best Threshold ---
print("Finding optimal decision threshold (for F1) on held-out validation set...")
y_val_probs = final_model.predict_proba(X_val_thresh)[:, 1]
thresholds = np.arange(0.35, 0.50, 0.001)
best_f1 = 0
best_threshold = 0.5

for thresh in thresholds:
    preds = (y_val_probs >= thresh).astype(int)
    f1 = f1_score(y_val_thresh, preds)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = thresh

print(f"Best Threshold for F1: {best_threshold:.4f} (with F1: {best_f1:.4f} on validation data)")

# --- 8. Final Evaluation ---
print("\n--- Evaluating final model on TEST set... ---")
y_test_probs = final_model.predict_proba(X_test)[:, 1]
y_final_pred = (y_test_probs >= best_threshold).astype(int)

final_f1 = f1_score(y_test, y_final_pred)
final_recall = recall_score(y_test, y_final_pred)
final_precision = precision_score(y_test, y_final_pred)

print("\n" + "="*50)
print("--- Final Model Results (Optuna + CV) ---")
print(f"   Best F1 Score:  {final_f1:.4f}")
print(f"   Recall:         {final_recall:.4f}")
print(f"   Precision:      {final_precision:.4f}")
print("="*50)

print("\n--- Full Classification Report (at Optimal F1 Threshold) ---")
print(classification_report(y_test, y_final_pred, digits=4))

print("\n--- Script Finished ---")

In [ ]:
train_df.value_counts